# A.I. Multi-Agent Study Assistant

This notebook demonstrates a multi-agent study planner: Scheduler Agent, Study Planner Agent, Progress Tracker Agent, and Resource Recommender Agent. The agents are powered by an LLM (Gemini 2.0 Pro recommended as described in the writeup). The notebook includes demo flows: `Plan my day`, `Generate weekly schedule`, `Adjust plan after missed session`.

**Notes:**
- Replace LLM calls with your provider's SDK (Gemini/ADK or OpenAI) and set credentials via environment variables.
- Do not commit API keys.


In [ ]:
# Install extras if needed (commented for Kaggle environment)
# !pip install --quiet openai==1.0.0 pydantic==1.10.7
# !pip install --quiet google-cloud-aiplatform


In [ ]:
import os
import json
import time
from typing import Dict, Any, List

class MemoryBank:
    def __init__(self):
        self.store = {}
    def get(self, key, default=None):
        return self.store.get(key, default)
    def set(self, key, value):
        self.store[key] = value
    def append_log(self, key, entry):
        self.store.setdefault(key, []).append(entry)

memory = MemoryBank()

def llm_call(prompt: str, model: str = "gemini-pro") -> str:
    # Placeholder for LLM call. Replace with real API calls.
    print(f"[LLM CALL — model={model}] Prompt length: {len(prompt)}")
    return "LLM_RESPONSE_PLACEHOLDER: " + prompt[:400]

import pprint
pp = pprint.PrettyPrinter(indent=2)


In [ ]:
class BaseAgent:
    def __init__(self, name: str, memory: MemoryBank):
        self.name = name
        self.memory = memory
    def act(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError

class SchedulerAgent(BaseAgent):
    def act(self, input_data):
        availability = {
            "monday": ["06:00-08:00","20:30-22:00"],
            "tuesday": ["06:00-08:00","20:30-22:00"],
            "wednesday": ["06:00-08:00","20:30-22:00"],
            "thursday": ["06:00-08:00","20:30-22:00"],
            "friday": ["06:00-08:00","20:30-22:00"],
            "saturday": ["09:00-12:00"],
            "sunday": ["09:00-12:00"]
        }
        self.memory.set('availability', availability)
        return {"availability": availability}

class StudyPlannerAgent(BaseAgent):
    def act(self, input_data):
        topics = input_data.get('topics', ['Topic A','Topic B','Topic C'])
        availability = input_data.get('availability', self.memory.get('availability'))
        plan = []
        days = list(availability.keys())
        t_idx = 0
        for d in days:
            slots = availability[d]
            for s in slots:
                if t_idx >= len(topics): break
                plan.append({"day": d, "slot": s, "topic": topics[t_idx]})
                t_idx += 1
            if t_idx >= len(topics): break
        self.memory.set('current_plan', plan)
        return {"plan": plan}

class ProgressTrackerAgent(BaseAgent):
    def act(self, input_data):
        completed = input_data.get('completed', [])
        memory_log = self.memory.get('progress', [])
        entry = {"time": time.time(), "completed": completed}
        memory_log.append(entry)
        self.memory.set('progress', memory_log)
        plan = self.memory.get('current_plan', [])
        completed_list = sum([e['completed'] for e in memory_log], [])
        remaining = [p for p in plan if p['topic'] not in completed_list]
        self.memory.set('remaining', remaining)
        return {"remaining": remaining}

class ResourceAgent(BaseAgent):
    def act(self, input_data):
        topic = input_data.get('topic')
        resources = [f"https://example.com/{topic.replace(' ','_')}_notes",
                     f"https://youtube.com/search?q={topic.replace(' ','+')}" ]
        self.memory.append_log('resources', {"topic": topic, "resources": resources})
        return {"resources": resources}

scheduler = SchedulerAgent('scheduler', memory)
planner = StudyPlannerAgent('planner', memory)
tracker = ProgressTrackerAgent('tracker', memory)
resource = ResourceAgent('resource', memory)


In [ ]:
# Demo flow
scheduler_out = scheduler.act({"user_calendar": {}, "constraints":{}})
pp.pprint(scheduler_out)

topics = ["Greedy Algorithms","Knapsack Problem","Job Sequencing","MST: Kruskal & Prim"]

planner_out = planner.act({"topics": topics, "availability": scheduler_out['availability'], "goals": {"exam_date": "2026-02-01"}})
pp.pprint(planner_out)

tracker_out = tracker.act({"completed": [topics[0]]})
pp.pprint(tracker_out)

res_out = resource.act({"topic": topics[1]})
pp.pprint(res_out)


In [ ]:
# Adjust plan after missing a session
tracker.act({"completed": []})
remaining = memory.get('remaining') or planner_out['plan']
remaining_topics = [r['topic'] for r in remaining] if remaining else topics[1:]
planner_out2 = planner.act({"topics": remaining_topics, "availability": scheduler_out['availability']})
pp.pprint(planner_out2)


In [ ]:
def evaluate_plan(plan: List[Dict[str,str]]) -> Dict[str,float]:
    coverage = len(set([p['topic'] for p in plan])) / max(1, len(topics))
    compactness = 1 - (len(plan) / (len(topics) + 2))
    return {"coverage": coverage, "compactness": compactness, "score": 0.6*coverage + 0.4*compactness}

pp.pprint(evaluate_plan(memory.get('current_plan')))


In [ ]:
# Simple logs
memory.append_log('logs', {"event": "demo_run", "time": time.time(), "notes": "Demo run completed"})
pp.pprint(memory.get('logs'))


**Next steps to convert this notebook to a production agent:**

1. Replace `llm_call` placeholder with proper Gemini 2.0 Pro / ADK calls.
2. Implement proper Prompt Templates for each agent and temperature tuning.
3. Add tool integrations: Google Search, Calendar API, and Code Execution where needed.
4. Add persistent memory (Memory Bank -> Datastore / Firestore) for long-term memory.
5. Add authentication and user interface (Flask/Streamlit) and deploy to Cloud Run for the bonus.